In [24]:
# Import libraries
from google import genai
from google.genai import types
from google.api_core import retry
from IPython.display import HTML, Markdown, display

# Set up secret credentials
from dotenv import load_dotenv 
load_dotenv()
import sys
import os
import requests
import os

In [25]:
# Error handling and retry logic not to worry about per-minute quota
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [26]:
# Set up your Google API key imported from .env file
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Initialize Gemini client
client = genai.Client(api_key=GOOGLE_API_KEY)

In [27]:
# Fetch the data for prompting
url = "https://www.ecb.europa.eu/press/press_conference/monetary-policy-statement/shared/pdf/ecb.ds250417~e613e58d41.es.pdf"
path = "data/ecb.pdf"
os.makedirs(os.path.dirname(path), exist_ok=True)
with open(path, "wb") as f:
    f.write(requests.get(url).content)

# Upload the file
document_file = client.files.upload(file=path)

PROMPTING 


1) *Zero-shot prompting* with parameters set for a slight creativity
2) *Few-shot prompting* with parameters set for a strict facticity
3) *Chain-of-thought prompting* with parameters set for the out-of-the-box ideas


In [28]:
# Zero-shot prompting
request = 'Tell me about the economic policy here.'

def summarise_doc(request: str) -> str:
  """Execute the request on the uploaded document."""
  # Set the temperature low to stabilise the output.
  config = types.GenerateContentConfig(temperature=0.2, top_p=0.95, top_k=30)
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=config,
      contents=[request, document_file],
  )

  return response.text

summary = summarise_doc(request)
Markdown(summary)

Here's a summary of the economic policy outlined in the document:

**Key Policy Decisions:**

*   **Interest Rate Cut:** The European Central Bank (ECB) Governing Council decided to lower the three key ECB interest rates by 25 basis points. This includes lowering the deposit facility rate to 2.25%, the main refinancing operations rate to 2.40%, and the marginal lending facility rate to 2.65%, effective April 23, 2025.
*   **Asset Purchase Programs:** The Asset Purchase Programme (APP) and Pandemic Emergency Purchase Programme (PEPP) portfolios are declining as the Eurosystem no longer reinvests principal payments from maturing securities.

**Rationale:**

*   **Inflation Target:** The primary goal is to ensure inflation stabilizes sustainably at the 2% medium-term target.
*   **Data-Dependent Approach:** The ECB will take a data-dependent, meeting-by-meeting approach to determine the appropriate monetary policy stance, considering the inflation outlook, economic and financial data, underlying inflation dynamics, and the strength of monetary policy transmission.
*   **Disinflation Progress:** The ECB believes the disinflation process is on track, with both headline and core inflation declining. They expect inflation to settle around the 2% target.
*   **Economic Outlook:** The euro area economy has shown some resilience, but the outlook has deteriorated due to rising trade tensions and increased uncertainty.

**Economic Assessment:**

*   **Economic Activity:** The economic outlook is clouded by uncertainty, with euro area exporters facing new trade barriers. Disruptions to international commerce, financial market tensions, and geopolitical uncertainty are weighing on business investment.
*   **Inflation:** Annual inflation edged down to 2.2% in March. Most indicators of underlying inflation point to a sustained return to the 2% target. Wages are gradually moderating.
*   **Risks:** Downside risks to economic growth have increased due to global trade tensions and geopolitical uncertainties. Increasing global trade disruptions add more uncertainty to the inflation outlook.
*   **Financial Conditions:** Risk-free interest rates have declined in response to escalating trade tensions. The euro has strengthened as investor sentiment has proven more resilient towards the euro area.

**Other Considerations:**

*   **Fiscal and Structural Policies:** The ECB emphasizes the importance of fiscal and structural policies to make the euro area economy more productive, competitive, and resilient.
*   **Digital Euro:** The ECB highlights the importance of rapidly establishing the legislative framework for the potential introduction of a digital euro.
*   **Transmission Protection Instrument:** The Transmission Protection Instrument is available to counter unwarranted, disorderly market dynamics that pose a serious threat to the transmission of monetary policy across all euro area countries.

In essence, the ECB is cautiously easing monetary policy to support economic growth while remaining committed to its inflation target. They are closely monitoring economic data and are prepared to adjust their policies as needed.


In [29]:
# Few-shot prompting
request = '''1. What is the ECB's stance on inflation in the latest report?
            2. Describe the ECB’s recent actions related to monetary policy.
            3. Summarize the ECB's view on economic growth in the euro area.
            Tell me about the economic policy here.'''

def summarise_doc(request: str) -> str:
  """Execute the request on the uploaded document."""
  # Set the temperature low to stabilise the output.
  config = types.GenerateContentConfig(temperature=0.1, top_p=0.9, top_k=20)
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=config,
      contents=[request, document_file],
  )

  return response.text

summary = summarise_doc(request)
Markdown(summary)

Here's a summary of the ECB's economic policy based on the provided document:

1.  **ECB's Stance on Inflation:**
    *   The ECB believes the disinflation process is "well on track."
    *   Both headline and core inflation declined in March.
    *   Most measures of underlying inflation suggest inflation will settle around the Governing Council's 2% medium-term target on a sustained basis.
    *   The ECB is determined to ensure that inflation stabilizes sustainably at its 2% medium-term target.

2.  **ECB's Recent Actions Related to Monetary Policy:**
    *   The Governing Council decided to lower the three key ECB interest rates by 25 basis points.
    *   Specifically, the deposit facility rate was lowered.
    *   The interest rates on the deposit facility, the main refinancing operations and the marginal lending facility will be decreased to 2.25%, 2.40% and 2.65% respectively, with effect from 23 April 2025.
    *   The APP and PEPP portfolios are declining at a measured and predictable pace, as the Eurosystem no longer reinvests the principal payments from maturing securities.

3.  **ECB's View on Economic Growth in the Euro Area:**
    *   The euro area economy has been building up some resilience against global shocks.
    *   The outlook for growth has deteriorated owing to rising trade tensions.
    *   Increased uncertainty is likely to reduce confidence among households and firms.
    *   The adverse and volatile market response to the trade tensions is likely to have a tightening impact on financing conditions.
    *   The economy is likely to have grown in the first quarter of the year, and manufacturing has shown signs of stabilisation.

**Overall Economic Policy Summary:**

The ECB's economic policy is focused on achieving its 2% medium-term inflation target while navigating a period of economic uncertainty. The ECB is taking a data-dependent, meeting-by-meeting approach to monetary policy, meaning it will adjust its policies based on incoming economic and financial data.

Key elements of the policy include:

*   **Interest Rate Adjustments:** Lowering interest rates to stimulate the economy and support the disinflation process.
*   **Quantitative Tightening:** Reducing the size of its balance sheet by not reinvesting proceeds from maturing assets in the APP and PEPP programs.
*   **Flexibility:** Being prepared to adjust all its instruments to ensure inflation stabilizes at the target and to preserve the smooth functioning of monetary policy transmission.
*   **Monitoring:** Closely monitoring economic and financial data, underlying inflation dynamics, and the strength of monetary policy transmission.
*   **Addressing Risks:** Being aware of downside risks to economic growth, particularly from trade tensions and geopolitical uncertainty.
*   **Fiscal and Structural Policies:** Emphasizing the importance of fiscal and structural policies to make the euro area economy more productive, competitive, and resilient.


In [30]:
# Chain-of-thought prompting
request = "What is the ECB's forecast for economic growth? Break it down into steps and think through the logic behind the ECB's predictions"

def summarise_doc(request: str) -> str:
  """Execute the request on the uploaded document."""
  # Set the temperature low to stabilise the output.
  config = types.GenerateContentConfig(temperature=0.9, top_p=0.99, top_k=40)
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=config,
      contents=[request, document_file],
  )

  return response.text

summary = summarise_doc(request)
Markdown(summary)

Okay, let's break down the ECB's economic growth forecast based on the provided document. The image displays the Combined monetary policy decisions and statement released by the European Central Bank (ECB) on April 17, 2025.

**1. Overall Assessment**

*   **Growth Deterioration:** The most critical takeaway is that the ECB believes the outlook for economic growth in the Euro area has **deteriorated**. The document explicitly states: "...the outlook for growth has deteriorated owing to rising trade tensions."
*   **Uncertainty:** The ECB emphasizes "exceptional uncertainty" surrounding the economic outlook.

**2. Factors Weighing on Growth (Downside Risks)**

*   **Trade Tensions:** Rising trade tensions are a primary concern. The ECB believes this will dampen exports and potentially drag down investment and consumption. The major escalation in global trade tensions and associated uncertainties will likely lower euro area growth by dampening exports, and it may drag down investment and consumption.
*   **Uncertainty:** Increased uncertainty is likely to reduce confidence among households and firms. Deteriorating financial market sentiment could lead to tighter financing conditions, increase risk aversion and make firms and households less willing to invest and consume.
*   **Financial Market Volatility:** Adverse and volatile market response to trade tensions is expected to tighten financing conditions, further weighing on the economic outlook.
*   **Geopolitical Tensions:** The Russia-Ukraine war and the Middle East conflict continue to be sources of uncertainty.
*   **Disruptions to International Commerce, financial market tensions and geopolitical uncertainty** are weighing on business investment.
*   **Consumers** are becoming more cautious about the future, they may hold back from spending.

**3. Balancing Factors (Potential Upside)**

*   **Resilience:** The ECB acknowledges that the Euro area economy has been building resilience against global shocks.
*   **First Quarter Growth:** The economy likely grew in the first quarter of the year.
*   **Manufacturing Stabilization:** Manufacturing has shown signs of stabilization.
*   **Strong Labor Market:** A strong labor market and higher real incomes are expected to underpin spending. Unemployment fell to 6.1 per cent in February, its lowest level since the launch of the euro.
*   **Fiscal Policy/Investment:** Policy initiatives at the national and EU levels to increase defense spending and infrastructure investment are expected to support manufacturing.
*   **Increase in defence and infrastructure spending** would add to growth.

**4. Logic Behind the ECB's Forecast**

The ECB's forecast is based on a combination of factors:

*   **Data-Driven:** The ECB emphasizes a "data-dependent and meeting-by-meeting approach." It will closely monitor incoming economic and financial data to assess the situation.
*   **Inflation Outlook:** The ECB's primary mandate is price stability. It is focused on ensuring inflation returns to its 2% medium-term target. The decision to lower interest rates is partly driven by the expectation that inflation will settle around the 2% target.
*   **Underlying Inflation Dynamics:** The ECB is analyzing the factors that influence underlying inflation, such as wage growth, profits, and productivity.
*   **Monetary Policy Transmission:** The ECB is assessing the effectiveness of its monetary policy in influencing the economy (i.e., how well interest rate changes translate into changes in borrowing costs, investment, and consumption).
*   **Risk Assessment:** The ECB explicitly identifies both downside and upside risks to the forecast. The downside risks appear to be a primary factor in the decision-making.

**5. In Summary**

The ECB's forecast appears cautiously pessimistic. While there are some positive signs in the Euro area economy, the ECB is concerned about the potential negative impacts of rising trade tensions, geopolitical uncertainty, and their influence on confidence and financial markets. Therefore, while they may have expected a period of growth in the Eurozone, the risks of the negative impacts of trade tensions and other geopolitical factors have caused them to revise their predictions. This has made them more cautious and has led them to lower interest rates in an effort to stimulate growth.

**Important Note:** This is an interpretation based *solely* on the provided document. A full understanding would require a review of additional data, forecasts, and statements from the ECB.